In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from numpy import mean

In [2]:
os.chdir(r'\Users\hossa084\Downloads\wildfire\WildFires-master')

In [ ]:

#creating a function to download a zip and unzip it in my desired location
def download_data(dw_url,destination):
    import wget
    import zipfile
    
    filename = wget.download(dw_url)
    print("Successfully downloaded", filename)
    
    with zipfile.ZipFile(destination + "/" + filename, 'r') as zip_ref:
        zip_ref.extractall(destination)

In [ ]:
#using my function
download_data('https://github.com/ouladsayadyounes/WildFires/archive/refs/heads/master.zip',r'\Users\hossa084\Downloads\wildfire\WildFires-master')

In [3]:
#opening the CSV and viewing part of it
wildfires = pd.read_csv('WildFires_DataSet.csv', float_precision='high')
wildfires.head()

,NDVI,LST,BURNED_AREA,CLASS
0,0.506782,14584.272727,4.692308,no_fire
1,0.522150,14655.833333,5.000000,no_fire
2,0.682284,14780.000000,5.000000,fire
3,0.120046,13298.500000,3.500000,no_fire
4,0.568734,14743.000000,5.000000,no_fire


In [4]:
#ML does not like classes that are a string
#change the classes to numbers
wildfires['CLASS'] = wildfires['CLASS'].map({'fire': 1, 'no_fire': 0})

In [5]:
#seperating the features from the labels
wildfire_features = wildfires.copy()
wildfire_labels = wildfire_features.pop('CLASS')
wildfire_labels.head()



0    0
1    0
2    1
3    0
4    0
Name: CLASS, dtype: int64

In [6]:
#create functions to normalize the data for the Neural Net
def normalize(x):
    x_med = x.median()
    x_std = x.std()
    x = ((x-x_med)/x_std)
    return x

def normalize_dataframe(pandas_df):
    pandas_df = pandas_df.apply(normalize)
    
    return pandas_df

In [7]:
wildfire_features = normalize_dataframe(wildfire_features) 
wildfire_features.head()
#now you might be thinking to yourself, "That's shome pretty shweet data"

,NDVI,LST,BURNED_AREA
0,-0.628978,-0.167546,-0.391111
1,-0.463460,0.027480,0.135948
2,1.261242,0.365876,0.135948
3,-4.794266,-3.671703,-2.433466
4,0.038267,0.265038,0.135948


In [8]:

#changing this from a series to values
wildfire_labels.values


array([0, 0, 1, ..., 1, 0, 1], dtype=int64)

In [9]:
wildfire_features = np.array(wildfire_features)
wildfire_features

array([[-6.28978449e-01, -1.67545939e-01, -3.91111091e-01],
       [-4.63459505e-01,  2.74804235e-02,  1.35947705e-01],
       [ 1.26124186e+00,  3.65875733e-01,  1.35947705e-01],
       ...,
       [ 5.15028005e-01,  2.88203790e-01, -3.28993790e+00],
       [ 5.30871209e-01, -2.82003661e+00,  7.16010091e-04],
       [-1.68922379e-01, -1.02464888e+00,  1.35947705e-01]])

In [10]:
#splitting my features and labels into train and test
wf_features_train, wf_features_test, wf_class_train, wf_class_test = model_selection.train_test_split(wildfire_features, wildfire_labels, test_size=0.3)

In [11]:
# Decision Tree method
from sklearn import tree
#model
DecisionTree = tree.DecisionTreeClassifier()
DecisionTree = DecisionTree.fit(wf_features_train, wf_class_train)


In [12]:
y_pred = DecisionTree.predict(wf_features_test)
y_pred[:10],wf_class_test[:10]

(array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 846     0
 57      0
 965     0
 364     0
 1573    0
 1480    1
 1093    0
 177     0
 35      0
 291     0
 Name: CLASS, dtype: int64)

In [13]:
# predict random values
print(DecisionTree.predict([[1.27242,0.385876,0.134487]]))
print(DecisionTree.predict([[1.461242,0.30876,0.135123]]))
print(DecisionTree.predict([[1.261242,0.365876,0.135948]]))
print(DecisionTree.predict([[1.461242,0.30876,0.13548]]))
print(DecisionTree.predict([[1.261242,0.365876,0.135948]]))

[1]
[1]
[1]
[1]
[1]


In [14]:
from sklearn.metrics import classification_report
# TREE model 
print(DecisionTree)
# make predictions
expected = wildfire_labels
predicted = DecisionTree.predict(wildfire_features)
# summarize the fit of the model
print(classification_report(expected, predicted))


DecisionTreeClassifier()
              precision    recall  f1-score   support

           0       0.96      0.95      0.96      1327
           1       0.85      0.87      0.86       386

    accuracy                           0.94      1713
   macro avg       0.90      0.91      0.91      1713
weighted avg       0.94      0.94      0.94      1713



In [15]:
#evaluate 
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(DecisionTree, wf_features_test, wf_class_test, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.740 (0.062)


In [ ]:
# decision tree graph
from sklearn.datasets import load_iris
from sklearn import tree
iris = load_iris()
wildfire_features, wildfire_labels = iris.data, iris.target
DecisionTree = tree.DecisionTreeClassifier()
DecisionTree = DecisionTree.fit(wildfire_features, wildfire_labels)
tree.plot_tree(DecisionTree)